## Импорт библиотек

In [1]:
# Для построения пайплайна обучения используется lightautoml, который конфликтуает с новой версией pandas. Поэтому
!pip uninstall pandas -y
!pip install --upgrade pip > installations.txt
!pip uninstall torch -y > installations.txt # конфликтует 
!pip install torch==2.0.0 > installations.txt
!pip install pandas==1.4.3 pyarrow yellowbrick polars transformers nltk gensim lightautoml fastparquet > installations.txt

Found existing installation: pandas 2.2.0
Uninstalling pandas-2.2.0:
  Successfully uninstalled pandas-2.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.
torchvision 0.16.0 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.


In [2]:
# for dataframe
import polars as pl
import numpy as np
import pyarrow as pa
import pandas as pd
import fastparquet

# for system
import os
import time
import sys

# for metric
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# for demention decrease
from sklearn.cluster import KMeans
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

# for common functions
from collections import OrderedDict
from collections import Counter
from kaggle_secrets import UserSecretsClient
from copy import deepcopy as copy
from typing import Tuple, List

# for monitoring of models
import wandb

# for machine learning
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.report.report_deco import ReportDeco

# my functions
sys.path.append('/kaggle/input/next-orders')
from my_functions import MarketDataProcessor

ModuleNotFoundError: No module named 'wandb'

## Препроцессинг данных

____________________________________________________
Использовался ноутбук с TPU и RAM 300GB, чтобы не испытывать проблем с памятью во время операционных обработок датасета.

In [9]:
# # tmp импорт в pandas
# raw = pd.read_csv('../input/sbermarket-internship-competition/train.csv')
sub = pd.read_csv('../input/sbermarket-internship-competition/sample_submission.csv', sep = ",")

# # Приведение столбца 'cart' к int
# raw['cart'] = raw['cart'].astype(int)

# mk_data = MarketDataProcessor(raw, sub)

# # Отбрасываем выбросы - юзеров с малым количеством товаров в истории их заказов
# filtered_raw, filtered_sub, proportion = mk_data.filter_train_data()
# print(f"Процент наблюдений, используемый для тренировки: {proportion:.2f}%")

### Polars ускоряет обработку данных
__________________________________________________
В итоге мы получаем pd датафрейм с усредненным рейтингом id во всем датасете и набор временных переменных, основанных на этой метрике (рейтинг)

In [15]:
# %%time
# mk_data.get_dummies_matrix()
# mk_data.generate_time_features()
# Train = mk_data.compile_dataset()

# Test = mk_data.compile_dataset(history_flag = 1)

### X / Y 

In [16]:
# x_cols = Train.select_dtypes(include=['int', 'int8', 'int32', 'uint32', 'uint8', 'float']).drop(columns = ['target'], axis = 1).columns.tolist()
# x_cols_pca = Train.select_dtypes(include=['int', 'int8', 'int32', 'uint32', 'uint8', 'float']).drop(columns = ['user_id', 'target'], axis = 1).columns.tolist()
# print(x_cols)
# y_cols = ['target']
# print(y_cols)

### Добавляем главные компоненты

In [17]:
# scaler = RobustScaler()

# s_Train = Train.copy()
# s_Test = Test.copy()
# s_Train[x_cols_pca] = scaler.fit_transform(s_Train[x_cols_pca])
# s_Test[x_cols_pca] = scaler.transform(s_Test[x_cols_pca])

# pca = PCA(n_components=10)

# s_Train_pca = pca.fit_transform(s_Train[x_cols])
# s_Test_pca = pca.transform(s_Test[x_cols])

# Train_pca = pd.DataFrame(s_Train_pca, columns=[f'pc{i+1}' for i in range(s_Train_pca.shape[1])])
# Test_pca = pd.DataFrame(s_Test_pca, columns=[f'pc{i+1}' for i in range(s_Train_pca.shape[1])])

# s_Train[Train_pca.columns.to_list()] = Train_pca
# s_Test[Test_pca.columns.to_list()] = Test_pca

### Кластеризация id(юзер-категория)

In [18]:
# from sklearn.cluster import KMeans
# from yellowbrick.cluster.elbow import kelbow_visualizer

# print('Elbow Method to determine the number of clusters for id and target:')
# kelbow_visualizer(KMeans(random_state=4), 
#                   Train_pca, 
#                   k=(2,21))


In [19]:
# from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=5)
# cluster = kmeans.fit_predict(Train_pca)
# test_cluster = kmeans.predict(Test_pca)
# s_Train["Clusters"] = cluster
# s_Test["Clusters"] = test_cluster

### Сохранение данных в parquet

In [20]:
# Train.to_parquet('Train.parquet', index=False)
# s_Train.to_parquet('s_Train.parquet', index=False)
# s_Test.to_parquet('s_Test.parquet', index=False)

In [3]:
# s_Train = pd.read_parquet('/kaggle/input/next-orders/s_Train (1).parquet')
s_Test = pd.read_parquet('/kaggle/input/next-orders/s_Test (1).parquet')

## LightAutoML пайплайн


In [4]:
def f1 (real, pred, **kwargs):
    return f1_score(real, (pred > 0.2).astype(int), **kwargs)

N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TARGET_NAME = 'target'
ROLES = {'target': TARGET_NAME, 'drop': ['id', 'user_id', 'category'], 'category': ['Clusters']}
TASK = Task('binary', metric = f1)

NameError: name 'Task' is not defined

In [5]:
Train_set, Valid_set = train_test_split(s_Train, test_size = TEST_SIZE,
                                        stratify = s_Train[TARGET_NAME], random_state = 23)
Train_set.reset_index(drop=True, inplace=True)
Valid_set.reset_index(drop=True, inplace=True)


In [6]:
del s_Train

In [7]:
real_estimators = ['total_order_in_cat_max',
 'ordered_mean',
 'pc2',
 'total_rating_cat_max',
 'rating_max',
 'rating_mean',
 'pc5',
 'weekday_mean',
 'pc6',
 'pc4',
 'pc10',
 'pc3',
 'pc1',
 'pc7',
 'pc8',
 'pc9',
 'rating_per_month_mean',
 'month_std',
 'month_mean',
 'rating_per_month_std',
 'weekday_std',
 'month_max',
 'ordered_std',
 'rating_std',
 'weekday_min',
 'month_min',
 't_rating_per_month_mean',
 'Clusters',
 'weekday_max',
 't_rating_per_month_sum',
 'tc_rating_per_month_mean',
 't_rating_per_month_median',
 't_rating_per_month_std',
 'tc_rating_per_month_sum',
 'ordered_max',
 'tc_rating_per_month_std']

In [2]:
Train_set[real_estimators + ['target']]

NameError: name 'Train_set' is not defined

In [ ]:
%%time 

automl = TabularAutoML(
    task = TASK,
    timeout = 2400,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
    general_params = {'use_algos': [['cb_tuned']]}
)


train_pred = automl.fit_predict(Train_set[real_estimators + ['target']], roles = ROLES, verbose = 2)

print('Score', "%.5f" % f1(Train_set.target, train_pred.data))
valid_pred = automl.predict(Valid_set[real_estimators + ['target']])
print('Score on out of folds validation', "%.5f" % f1(Valid_set.target, valid_pred.data))

[07:18:35] Stdout logging level is INFO2.
[07:18:35] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[07:18:35] Task: binary

[07:18:35] Start automl preset with listed constraints:
[07:18:35] - time: 3600.00 seconds
[07:18:35] - CPU: 4 cores
[07:18:35] - memory: 16 GB

[07:18:35] Train data shape: (14096000, 37)



/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py:37: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


[07:18:58] Layer 1 train process start. Time left 3576.14 secs


In [ ]:
# print(automl.create_model_str_desc())

In [ ]:
# fast_fi = automl.get_feature_scores('fast', silent=False)
# fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (30, 10), grid = True)

In [5]:
import joblib

# joblib.dump(automl, 'automl_lgbm.pkl')
automl=joblib.load('/kaggle/input/next-orders/automl_lgbm.pkl')

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
valid_pred = automl.predict(Valid_set)
print('Score on out of folds validation', "%.5f" % f1(Valid_set.target, valid_pred.data))

### Нахождение лучшего порогового значения для сепарации 0 и 1

In [ ]:
best_score = 0
for i in np.arange(0.01, 1.0, 0.01):
    score = f1 = f1_score(Valid_set.target, (valid_pred.data > i).astype(int))
    if score > best_score:
        best_score = score
        proba_split = i

print('At i =', "%.2f" % proba_split,'score is : ' "%.5f" % best_score)

In [10]:
predictions = automl.predict(s_Test[real_estimators])
s_Test['target'] = (predictions.data > 0.21).astype(int)
submit = pd.merge(sub['id'], s_Test[['id', 'target']], on='id')

## Подготовка сабмита

In [11]:
import csv

with open('submission_lgbm_01.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(submit.columns)
    for row in submit.values:
        csvwriter.writerow(row)

##  Dense

In [ ]:
# default_lama_params = {
#     "task": task,
#     "timeout": TIMEOUT,
#     "cpu_limit": N_THREADS,
#     "reader_params": {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
# }

# default_nn_params = {
#     "bs": 512, "num_workers": 4, "path_to_save": None, "n_epochs": 10, "freeze_defaults": True
# }

In [ ]:
# import torch.nn as nn
# from lightautoml.automl.presets.tabular_presets import TabularAutoML

# Train_set, Valid_set = train_test_split(s_Train, test_size = TEST_SIZE,
#                                         stratify = s_Train[TARGET_NAME], random_state = 23)
# Train_set.reset_index(drop=True, inplace=True)
# Valid_set.reset_index(drop=True, inplace=True)

# automl = TabularAutoML(
#     **default_lama_params,
#     general_params={"use_algos": [["denselight_tuned"]]},
#     nn_params={
#         **default_nn_params,
#         "n_epochs": 5,
#         "tuning_params": {
#             "max_tuning_iter": 5,
#             "max_tuning_time": 500,
#             "fit_on_holdout": True
#         }
#     },
# )

# train_pred = automl.fit_predict(Train_set[real_estimators + ['target']], roles = ROLES, verbose = 2)

# print('Score', "%.5f" % f1(Train_set.target, train_pred.data))
# valid_pred = automl.predict(Valid_set[real_estimators + ['target']])
# print('Score on out of folds validation', "%.5f" % f1(Valid_set.target, valid_pred.data))